In [ ]:
!mkdir -p rawdata && cd rawdata && wget https://synchronouscap.s3.eu-west-3.amazonaws.com/obchallenge/ETHUSDT-all.zip 

--2023-11-28 22:54:51--  https://synchronouscap.s3.eu-west-3.amazonaws.com/obchallenge/ETHUSDT-all.zip
Resolving synchronouscap.s3.eu-west-3.amazonaws.com (synchronouscap.s3.eu-west-3.amazonaws.com)... 52.95.154.86, 3.5.225.182
Connecting to synchronouscap.s3.eu-west-3.amazonaws.com (synchronouscap.s3.eu-west-3.amazonaws.com)|52.95.154.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8364338229 (7,8G) [application/zip]
Saving to: ‘ETHUSDT-all.zip’

ETHUSDT-all.zip      11%[=>                  ] 883,76M  14,9MB/s    eta 7m 50s 

In [ ]:
!pwd

In [ ]:
# list all files in archive
!unzip -l rawdata/ETHUSDT-all.zip

In [ ]:
!mkdir -p splitted
# list all files             extract only file name            run bash shell for each file in parallel        unzip just selected file and compress it to gz
!time unzip -l rawdata/ETHUSDT-all.zip | egrep '([0-9A-Za-z\\-]+.csv)' -o | xargs -P 8 -I{} bash -c 'echo processing {} && unzip -p rawdata/ETHUSDT-all.zip {} | gzip > splitted/{}.gz'

In [ ]:
# parse number of columns per line
!ls ./splitted/ | grep '\.csv\.gz'| time xargs -P 8 -I{} bash -c "zcat splitted/{} | python3 -c 'import sys; print(set(map(lambda row: len(row.split(\",\")), sys.stdin)))'" | tee > rows.txt

In [ ]:
# check each row has same number of columns
!cat rows.txt | sort | uniq -c

In [ ]:
# each file contains lines with exactly 4004 columns
#     0                        1-2             3-2002  2003-4002, 4003
# unix time in miliseconds, unknown costants, eth-usd, coef,    <empty>

In [ ]:
# extract timestamps, some lines can have BOM char as prefix
!zcat splitted/*.csv.gz | time python3 -c  'import sys; list(map(lambda row: print(int(row.strip().strip("\ufeff")[:13])), sys.stdin))' > tm.txt

In [ ]:
!wc -l tm.txt

In [ ]:
with open('tm.txt') as f:
    values = [int(v) for v in f]
len(values)

In [ ]:
diffs = []
for i, val in enumerate(values):
    if i == 0:
        diffs.append(0)
        continue
    diffs.append(val - values[i-1])

diffs[:10]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(diffs[1:])
df.describe(percentiles=[i / 100 for i in range(1, 100)])

In [ ]:
# min distance in MS between frames, also no negative dist, data is ordered
st = list(sorted(diffs))
st[:10]

In [ ]:
# max distance in MS between frames
st[-100:]

In [ ]:
!cd .. && cargo build --release

In [ ]:
!du -sch splitted/*

In [ ]:
!

In [ ]:
!mkdir -p sample_5s && ls splitted/ | grep '\.csv\.gz' | time xargs -P 8 -I{} bash -c 'echo processing {} && zcat splitted/{} | ../target/release/parse-ethusd | gzip > sample_5s/{}'